# DX 704 Week 4 Project

This week's project will test the learning speed of linear contextual bandits compared to unoptimized approaches.
You will start with building a preference data set for evaluation, and then implement different variations of LinUCB and visualize how fast they learn the preferences.


The full project description, a template notebook and supporting code are available on GitHub: [Project 4 Materials](https://github.com/bu-cds-dx704/dx704-project-04).


## Example Code

You may find it helpful to refer to these GitHub repositories of Jupyter notebooks for example code.

* https://github.com/bu-cds-omds/dx601-examples
* https://github.com/bu-cds-omds/dx602-examples
* https://github.com/bu-cds-omds/dx603-examples
* https://github.com/bu-cds-omds/dx704-examples

Any calculations demonstrated in code examples or videos may be found in these notebooks, and you are allowed to copy this example code in your homework answers.

## Part 1: Collect Rating Data

The file "recipes.tsv" in this repository has information about 100 recipes.
Make a new file "ratings.tsv" with two columns, recipe_slug (from recipes.tsv) and rating.
Populate the rating column with values between 0 and 1 where 0 is the worst and 1 is the best.
You can assign these ratings however you want within that range, but try to make it reflect a consistent set of preferences.
These could be your preferences, or a persona of your choosing (e.g. chocolate lover, bacon-obsessed, or sweet tooth).
Make sure that there are at least 10 ratings of zero and at least 10 ratings of one.


Hint: You may find it more convenient to assign raw ratings from 1 to 5 and then remap them as follows.

`ratings["rating"] = (ratings["rating_raw"] - 1) * 0.25`

Submit "ratings.tsv" in Gradescope.

In [3]:
! pip install pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 47.1 MB/s  0:00:006m0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.6/16.6 MB 46.7 MB/s  0:00:006m0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [pandas]2m1/2 [pandas]

[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [4]:
import pandas as pd
import random

recipes = pd.read_csv("recipes.tsv", sep="\t")
ratings = []

for index, row in recipes.iterrows():
    name = row["recipe_slug"].lower()
    
    # Chocolate lover
    if "chocolate" in name or "cake" in name or "cookie" in name or "brownie" in name or "dessert" in name:
        rating = 1.0
    elif "salad" in name or "kale" in name or "tofu" in name or "broccoli" in name:
        rating = 0.0
    else:
        rating = round(random.uniform(0, 1), 2)  # random rating between 0 and 1
    
    ratings.append(rating)


zero_count = ratings.count(0.0)
one_count = ratings.count(1.0)

i = 0
while zero_count < 10:
    ratings[i] = 0.0
    zero_count += 1
    i += 1

j = len(ratings) - 1
while one_count < 10:
    ratings[j] = 1.0
    one_count += 1
    j -= 1

ratings_df = pd.DataFrame({
    "recipe_slug": recipes["recipe_slug"],
    "rating": ratings
})

ratings_df.to_csv("ratings.tsv", sep="\t", index=False)

## Part 2: Construct Model Input

Use your file "ratings.tsv" combined with "recipe-tags.tsv" to create a new file "features.tsv" with a column recipe_slug, a column bias which is hard-coded to one, and a column for each tag that appears in "recipe-tags.tsv".
The tag column in this file should be a 0-1 encoding of the recipe tags for each recipe.
[Pandas reshaping function methods](https://pandas.pydata.org/docs/user_guide/reshaping.html) may be helpful.

The bias column will make later LinUCB calculations easier since it will just be another dimension.

Hint: For later modeling steps, it will be important to have the feature data (inputs) and the rating data (target outputs) in the same order.
It is highly recommended to make sure that "features.tsv" and "ratings.tsv" have the recipe slugs in the same order.

Submit "features.tsv" in Gradescope.

In [8]:
ratings = pd.read_csv("ratings.tsv", sep="\t")

tags = pd.read_csv("recipe-tags.tsv", sep="\t")

tag_matrix = pd.crosstab(tags["recipe_slug"], tags["recipe_tag"])
tag_matrix = (tag_matrix > 0).astype(int)

tag_matrix = tag_matrix.reindex(ratings["recipe_slug"]).fillna(0).astype(int)

features = pd.DataFrame({
    "recipe_slug": ratings["recipe_slug"],
    "bias": 1
})

features = pd.concat([features, tag_matrix.reset_index(drop=True)], axis=1)

features.to_csv("features.tsv", sep="\t", index=False)

print("Same order as ratings?", (features["recipe_slug"] == ratings["recipe_slug"]).all())
print("Shape:", features.shape)
print(features.head())

Same order as ratings? True
Shape: (100, 298)
        recipe_slug  bias  alfredo  almond  american  appetizer  appetizers  \
0           falafel     1        0       0         0          1           0   
1        spamburger     1        0       0         0          0           0   
2  bacon-fried-rice     1        0       0         0          0           0   
3   chicken-fingers     1        0       0         0          1           0   
4       apple-crisp     1        0       0         0          0           0   

   apple  asiancuisine  asparagus  ...  udonnoodles  vanilla  vanillaicecream  \
0      0             0          0  ...            0        0                0   
1      0             0          0  ...            0        0                0   
2      0             0          0  ...            0        0                0   
3      0             0          0  ...            0        0                0   
4      1             0          0  ...            0        0              

## Part 3: Linear Preference Model

Use your feature and rating files to build a ridge regression model with ridge regression's regularization parameter $\alpha$ set to 1.


Hint: If you are using scikit-learn modeling classes, you should use `fit_intercept=False` since that intercept value will be redundant with the bias coefficient.

Hint: The estimate component of the bounds should match the previous estimate, so you should be able to just focus on the variance component of the bounds now.

In [11]:
!pip install scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 45.3 MB/s  0:00:006m0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.0/35.0 MB 51.0 MB/s  0:00:006m0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4/4 [scikit-learn] [scikit-learn]

[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [12]:
# YOUR CHANGES HERE

from sklearn.linear_model import Ridge

features = pd.read_csv("features.tsv", sep="\t")
ratings = pd.read_csv("ratings.tsv", sep="\t")

X = features.drop(columns=["recipe_slug"])

y = ratings["rating"]

model = Ridge(alpha=1, fit_intercept=False)

model.fit(X, y)

coefficients = pd.DataFrame({
    "feature": X.columns,
    "coefficient": model.coef_
})

print(coefficients.head())

     feature  coefficient
0       bias     0.403319
1    alfredo     0.102896
2     almond     0.128822
3   american    -0.059271
4  appetizer    -0.073104


Save the coefficients of this model in a file "model.tsv" with columns "recipe_tag" and "coefficient".
Do not add anything for the `intercept_` attribute of a scikit-learn model; this will be covered by the coefficient for the bias column added in part 2.

In [ ]:
# YOUR CHANGES HERE

...

Submit "model.tsv" in Gradescope.

## Part 4: Recipe Estimates

Use the recipe model to estimate the score of every recipe.
Save these estimates to a file "estimates.tsv" with columns recipe_slug and score_estimate.

In [ ]:
# YOUR CHANGES HERE

...

Submit "estimates.tsv" in Gradescope.

## Part 5: LinUCB Bounds

Calculate the upper bounds of LinUCB using data corresponding to trying every recipe once and receiving the rating in "ratings.tsv" as the reward.
Keep the ridge regression regularization parameter at 1, and set LinUCB's $\alpha$ parameter to 2.
Save these upper bounds to a file "bounds.tsv" with columns recipe_slug and score_bound.

In [ ]:
# YOUR CHANGES HERE

...

Submit "bounds.tsv" in Gradescope.

## Part 6: Make Online Recommendations

Implement LinUCB to make 100 recommendations starting with no data and using the same parameters as in part 5.
One recommendation should be made at a time and you can break ties arbitrarily.
After each recommendation, use the rating from part 1 as the reward to update the LinUCB data.
Record the recommendations made in a file "recommendations.tsv" with columns "recipe_slug", "score_bound", and "reward".
The rows in this file should be in the same order as the recommendations were made.

Hint: do not remove recipes after each recommendation.
Repeating recommendations is expected.

In [ ]:
# YOUR CHANGES HERE

...

Submit "recommendations.tsv" in Gradescope.

## Part 7: Acknowledgments

Make a file "acknowledgments.txt" documenting any outside sources or help on this project.
If you discussed this assignment with anyone, please acknowledge them here.
If you used any libraries not mentioned in this module's content, please list them with a brief explanation what you used them for.
If you used any generative AI tools, please add links to your transcripts below, and any other information that you feel is necessary to comply with the generative AI policy.
If no acknowledgements are appropriate, just write none in the file.


Submit "acknowledgments.txt" in Gradescope.

## Part 8: Code

Please submit a Jupyter notebook that can reproduce all your calculations and recreate the previously submitted files.


Submit "project.ipynb" in Gradescope.